In [ ]:
just_checking_integrity=False

In [ ]:
import gc
import pandas as pd
import scipy.sparse as sps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
valid=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [ ]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["Graph"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


#del dataset_max
del df_feats
del df_max

0
loading Graph
9381571
loaded Graph


In [ ]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/train/candidates.csv")
# if just_checking_integrity:
#     submission_df["score"]=submission_df["score"].apply(lambda x:x.replace("-inf","0")).apply(lambda x:x.replace("+inf","0")).apply(lambda x:x.replace("inf","0"))
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# else:
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [ ]:
# submission_df.head()

In [ ]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/train/candidates.csv")
# if just_checking_integrity:
#     submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# else:
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [ ]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/train/candidates.csv")
if just_checking_integrity:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
else:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [ ]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")

dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")


In [ ]:
if just_checking_integrity:
    dataset=dataset[:int(0.5*len(dataset))]

In [ ]:
del dataset_max
# del submission_df
# del submission_df_tr

In [ ]:
import numpy as np
sessions=np.unique(dataset["Session_Id"])
train=pd.DataFrame()
train["Session_Id"]=sessions
if just_checking_integrity:
    train["Item_ID"]=bought[:len(sessions)]
else:
    train["Item_ID"]=bought
train["target"]=1

In [ ]:
gc.collect()

451

In [ ]:
dataset=dataset.merge(train,on=["Session_Id","Item_ID"],how="left")
dataset.fillna(0,inplace=True)
gc.collect()

0

In [ ]:
dataset["target"]=dataset["target"].astype("int8")
gc.collect()

0

In [ ]:
sessions_containing_bought_among_candidates=dataset[["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

dataset=dataset[dataset['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

In [ ]:
NHITS=len(dataset['Session_Id'].unique())
HITRATE=NHITS/81600 #approximate value
HITRATE

0.6230882352941176

In [ ]:
dataset.head(20)

,Session_Id,Item_ID,Score_Graph,score_LSTM,rank_LSTM,Max_Score_Graph,target
107,918383,55,0.005355,0.001339,47.0,91.79825,0
108,918383,189,0.020602,0.000000,0.0,91.79825,0
109,918383,260,0.003595,0.000000,0.0,91.79825,0
110,918383,357,0.003286,0.000000,0.0,91.79825,0
111,918383,379,0.004070,0.000850,73.0,91.79825,0
112,918383,421,0.013832,0.000775,80.0,91.79825,0
113,918383,506,0.003990,0.000000,0.0,91.79825,0
114,918383,531,0.006495,0.000982,65.0,91.79825,0
115,918383,548,0.044912,0.000748,87.0,91.79825,0
116,918383,609,0.004670,0.000000,0.0,91.79825,0


In [ ]:
dataset

,Session_Id,Item_ID,Score_Graph,score_LSTM,rank_LSTM,Max_Score_Graph,target
107,918383,55,0.005355,0.001339,47.0,91.798250,0
108,918383,189,0.020602,0.000000,0.0,91.798250,0
109,918383,260,0.003595,0.000000,0.0,91.798250,0
110,918383,357,0.003286,0.000000,0.0,91.798250,0
111,918383,379,0.004070,0.000850,73.0,91.798250,0
...,...,...,...,...,...,...,...
14333977,999999,1092,0.000000,0.001129,96.0,12.826448,0
14333978,999999,1513,0.000000,0.001127,97.0,12.826448,0
14333979,999999,827,0.000000,0.001115,98.0,12.826448,0
14333980,999999,3645,0.000000,0.001111,99.0,12.826448,0


In [ ]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [ ]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

## K-Fold for All tradition models + LSTM models

In [ ]:
from sklearn.model_selection import KFold
import xgboost as xgb
import os
import time

start_time = time.time()

k = 10

score = 0
kf = KFold(n_splits=k)
index = 0
for train_index, valid_index in kf.split(train):
    train_sessions = train["Session_Id"][train_index].tolist()
    valid_sessions = train["Session_Id"][valid_index].tolist()

    train_set = dataset[dataset["Session_Id"].isin(train_sessions)]
    valid_set = dataset[dataset["Session_Id"].isin(valid_sessions)]

    train_set = train_set.sort_values("Session_Id")
    valid_set = valid_set.sort_values("Session_Id")

    qids_train = train_set.groupby("Session_Id").size().values
    qids_valid = valid_set.groupby("Session_Id").size().values

    X_train = train_set.drop(["target", "Session_Id", "Item_ID"], axis=1)
    y_train = train_set["target"]

    X_valid = valid_set.drop(["target", "Session_Id", "Item_ID"], axis=1)
    y_valid = valid_set["target"]

    model = xgb.XGBRanker(
        objective="rank:pairwise",
        n_estimators=2000,
        learning_rate=0.05,
        max_depth=6,
        reg_alpha=1e-4,
        subsample=0.8,
        min_child_weight=400
    )

    model.fit(
        X_train, y_train, group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        verbose=True,
        early_stopping_rounds=20,
        eval_metric=["map", "ndcg"]
    )

    model.save_model(f'./drive/MyDrive/Recommendation_system/rankers/xgboost/graph_and_lstm/model_{index}.xgb')

    # Get maximum validation score
    score += max(model.evals_result_["validation_0"]["map"])
    index += 1

end_time = time.time()
elapsed_time = end_time - start_time


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-map:0.25842	validation_0-ndcg:0.34867
[1]	validation_0-map:0.26062	validation_0-ndcg:0.35048
[2]	validation_0-map:0.26092	validation_0-ndcg:0.35154
[3]	validation_0-map:0.26065	validation_0-ndcg:0.35159
[4]	validation_0-map:0.26139	validation_0-ndcg:0.35270
[5]	validation_0-map:0.26166	validation_0-ndcg:0.35286
[6]	validation_0-map:0.26129	validation_0-ndcg:0.35256
[7]	validation_0-map:0.26127	validation_0-ndcg:0.35280
[8]	validation_0-map:0.26239	validation_0-ndcg:0.35349
[9]	validation_0-map:0.26174	validation_0-ndcg:0.35324
[10]	validation_0-map:0.26153	validation_0-ndcg:0.35276
[11]	validation_0-map:0.26169	validation_0-ndcg:0.35311
[12]	validation_0-map:0.26074	validation_0-ndcg:0.35248
[13]	validation_0-map:0.26082	validation_0-ndcg:0.35227
[14]	validation_0-map:0.26075	validation_0-ndcg:0.35243
[15]	validation_0-map:0.26119	validation_0-ndcg:0.35301
[16]	validation_0-map:0.26128	validation_0-ndcg:0.35288
[17]	validation_0-map:0.26104	validation_0-ndcg:0.35284
[1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:34:17] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-map:0.26574	validation_0-ndcg:0.35791
[1]	validation_0-map:0.27070	validation_0-ndcg:0.36271
[2]	validation_0-map:0.27104	validation_0-ndcg:0.36357
[3]	validation_0-map:0.27228	validation_0-ndcg:0.36488
[4]	validation_0-map:0.27277	validation_0-ndcg:0.36550
[5]	validation_0-map:0.27363	validation_0-ndcg:0.36635
[6]	validation_0-map:0.27384	validation_0-ndcg:0.36626
[7]	validation_0-map:0.27388	validation_0-ndcg:0.36652
[8]	validation_0-map:0.27175	validation_0-ndcg:0.36509
[9]	validation_0-map:0.27286	validation_0-ndcg:0.36563
[10]	validation_0-map:0.27254	validation_0-ndcg:0.36600
[11]	validation_0-map:0.27323	validation_0-ndcg:0.36613
[12]	validation_0-map:0.27311	validation_0-ndcg:0.36618
[13]	validation_0-map:0.27285	validation_0-ndcg:0.36614
[14]	validation_0-map:0.27252	validation_0-ndcg:0.36559
[15]	validation_0-map:0.27298	validation_0-ndcg:0.36630
[16]	validation_0-map:0.27269	validation_0-ndcg:0.36623
[17]	validation_0-map:0.27307	validation_0-ndcg:0.36638
[1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:34:46] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:38:28] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-map:0.26678	validation_0-ndcg:0.35453
[1]	validation_0-map:0.27531	validation_0-ndcg:0.36108
[2]	validation_0-map:0.27482	validation_0-ndcg:0.36082
[3]	validation_0-map:0.27505	validation_0-ndcg:0.36188
[4]	validation_0-map:0.27524	validation_0-ndcg:0.36212
[5]	validation_0-map:0.27505	validation_0-ndcg:0.36213
[6]	validation_0-map:0.27443	validation_0-ndcg:0.36195
[7]	validation_0-map:0.27427	validation_0-ndcg:0.36153
[8]	validation_0-map:0.27467	validation_0-ndcg:0.36193
[9]	validation_0-map:0.27500	validation_0-ndcg:0.36239
[10]	validation_0-map:0.27455	validation_0-ndcg:0.36218
[11]	validation_0-map:0.27427	validation_0-ndcg:0.36206
[12]	validation_0-map:0.27488	validation_0-ndcg:0.36257
[13]	validation_0-map:0.27466	validation_0-ndcg:0.36239
[14]	validation_0-map:0.27495	validation_0-ndcg:0.36283
[15]	validation_0-map:0.27410	validation_0-ndcg:0.36174
[16]	validation_0-map:0.27482	validation_0-ndcg:0.36198
[17]	validation_0-map:0.27457	validation_0-ndcg:0.36215
[1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:39:14] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-map:0.25308	validation_0-ndcg:0.33861
[1]	validation_0-map:0.25687	validation_0-ndcg:0.34273
[2]	validation_0-map:0.25960	validation_0-ndcg:0.34496
[3]	validation_0-map:0.25846	validation_0-ndcg:0.34423
[4]	validation_0-map:0.25921	validation_0-ndcg:0.34546
[5]	validation_0-map:0.25917	validation_0-ndcg:0.34543
[6]	validation_0-map:0.25968	validation_0-ndcg:0.34555
[7]	validation_0-map:0.25988	validation_0-ndcg:0.34595
[8]	validation_0-map:0.25910	validation_0-ndcg:0.34549
[9]	validation_0-map:0.25943	validation_0-ndcg:0.34570
[10]	validation_0-map:0.25900	validation_0-ndcg:0.34555
[11]	validation_0-map:0.25948	validation_0-ndcg:0.34582
[12]	validation_0-map:0.25836	validation_0-ndcg:0.34504
[13]	validation_0-map:0.25873	validation_0-ndcg:0.34552
[14]	validation_0-map:0.25882	validation_0-ndcg:0.34602
[15]	validation_0-map:0.25911	validation_0-ndcg:0.34614
[16]	validation_0-map:0.25966	validation_0-ndcg:0.34686
[17]	validation_0-map:0.25952	validation_0-ndcg:0.34696
[1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:40:34] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [ ]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.2807679827167322


In [ ]:
print("normalized score ", final_performcance*HITRATE)

normalized score  0.17494322687805797


In [ ]:
print("K-Fold execution time:", elapsed_time, "seconds")

K-Fold execution time: 408.89262533187866 seconds
